# HW 5 Question 2

# Setup

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import torch
import torch.optim as optim
torch.set_printoptions(edgeitems=2, linewidth=75)

## define model and functions

In [2]:
percent_for_validation = 0.2

In [3]:
def model(feature, w5, w4, w3, w2, w1, b):
    return feature[4] * w5 + feature[3] * w4 + feature[2] * w3 + feature[1] * w2 + feature[0] * w1 + b

In [4]:
def loss_function(predicted, actual):
    squared_diffs = (predicted - actual)**2
    return squared_diffs.mean()

In [5]:
rates_to_learn_at = [1/x for x in [5, 10, 100, 1000, 10000, 100000]]
rates_to_learn_at

[0.2, 0.1, 0.01, 0.001, 0.0001, 1e-05]

## Data import

In [6]:
housing_df = pd.DataFrame(pd.read_csv('Housing.csv'))
housing_df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [7]:
print(f"shape = {np.shape(housing_df)}")

# creates a list of all variables from the column names
feature_list = list( housing_df.columns )

print(f"features are: {feature_list}")

shape = (545, 13)
features are: ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'furnishingstatus']


In [8]:
# Maps to turn categorys into numbers 
def boolean_map(x):
    return x.map({'yes': 1 , 'no': 0})
def furnish_map(x):
    return x.map({'furnished': 1 , 'semi-furnished': 0.5 , 'unfurnished': 0})

# Extracts the yes and no column names
binary_vars = [*feature_list[5:10], feature_list[11]]
print(f"binary vars = {binary_vars}")

# Extracts the furnishing column names
furnish_vars = [feature_list[12]]
print(f"furnish vars = {furnish_vars}")

# Extracts the column names that are actual values
valued_vars = feature_list.copy()
[valued_vars.remove( item ) for item in binary_vars]
[valued_vars.remove( item ) for item in furnish_vars]
print(f"value vars = {valued_vars}")

binary vars = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
furnish vars = ['furnishingstatus']
value vars = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking']


In [9]:
x_df = housing_df.copy()

## scale data
scaler = StandardScaler()
# scaler = MinMaxScaler()

x_df[valued_vars] = scaler.fit_transform(x_df[valued_vars])

## map text values
x_df[binary_vars] = x_df[binary_vars].apply(boolean_map)
x_df[furnish_vars] = x_df[furnish_vars].apply(furnish_map)

## make y_df
y_df = x_df.pop('price')


In [10]:
# input_values = valued_vars.copy()
# input_values.remove('price')


# x_df = x_df[input_values]

In [11]:
x_df.head()

,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,1.046726,1.403419,1.421812,1.378217,1,0,0,0,1,1.517692,1,1.0
1,1.757010,1.403419,5.405809,2.532024,1,0,0,0,1,2.679409,0,1.0
2,2.218232,0.047278,1.421812,0.224410,1,0,1,0,0,1.517692,1,0.5
3,1.083624,1.403419,1.421812,0.224410,1,0,1,0,1,2.679409,1,1.0
4,1.046726,1.403419,-0.570187,0.224410,1,1,1,0,1,1.517692,0,1.0


In [12]:
y_df.head()

0    4.566365
1    4.004484
2    4.004484
3    3.985755
4    3.554979
Name: price, dtype: float64

In [13]:
# Remove unwanted data
for item in [*binary_vars, *furnish_vars] :
    x_df.pop(item)
x_df.head()

,area,bedrooms,bathrooms,stories,parking
0,1.046726,1.403419,1.421812,1.378217,1.517692
1,1.757010,1.403419,5.405809,2.532024,2.679409
2,2.218232,0.047278,1.421812,0.224410,1.517692
3,1.083624,1.403419,1.421812,0.224410,2.679409
4,1.046726,1.403419,-0.570187,0.224410,1.517692


## convert data to tensor and test accessing

In [14]:
# Convert data frame to tensor

x = torch.tensor(x_df.values)
y = torch.tensor(y_df.values)

In [15]:
print(x.shape)
print("\n")
x

torch.Size([545, 5])




tensor([[ 1.0467,  1.4034,  ...,  1.3782,  1.5177],
        [ 1.7570,  1.4034,  ...,  2.5320,  2.6794],
        ...,
        [-1.0334,  0.0473,  ..., -0.9294, -0.8057],
        [-0.5998,  0.0473,  ...,  0.2244, -0.8057]], dtype=torch.float64)

In [16]:
print(y.shape)
print("\n")
y

torch.Size([545])




tensor([ 4.5664e+00,  4.0045e+00,  4.0045e+00,  3.9858e+00,  3.5550e+00,
         3.2553e+00,  2.8807e+00,  2.8807e+00,  2.7309e+00,  2.6934e+00,
         2.6934e+00,  2.6297e+00,  2.4312e+00,  2.3938e+00,  2.3938e+00,
         2.3188e+00,  2.3188e+00,  2.2439e+00,  2.2065e+00,  2.1877e+00,
         2.1315e+00,  2.0941e+00,  2.0754e+00,  2.0754e+00,  2.0379e+00,
         2.0192e+00,  1.9780e+00,  1.9443e+00,  1.9443e+00,  1.9443e+00,
         1.9443e+00,  1.9443e+00,  1.8881e+00,  1.8319e+00,  1.7944e+00,
         1.7735e+00,  1.7532e+00,  1.7195e+00,  1.7101e+00,  1.6820e+00,
         1.6633e+00,  1.6446e+00,  1.5697e+00,  1.5697e+00,  1.4947e+00,
         1.4947e+00,  1.4760e+00,  1.4573e+00,  1.4386e+00,  1.4198e+00,
         1.4198e+00,  1.4198e+00,  1.3824e+00,  1.3824e+00,  1.3824e+00,
         1.3824e+00,  1.3786e+00,  1.3262e+00,  1.3075e+00,  1.3075e+00,
         1.2700e+00,  1.2325e+00,  1.2325e+00,  1.2138e+00,  1.1951e+00,
         1.1576e+00,  1.1576e+00,  1.1389e+00,  1.1

In [17]:
x[1][0]

tensor(1.7570, dtype=torch.float64)

In [18]:
x[0][1]

tensor(1.4034, dtype=torch.float64)

In [19]:
x[:][0]

tensor([1.0467, 1.4034, 1.4218, 1.3782, 1.5177], dtype=torch.float64)

In [20]:
x[0][:]

tensor([1.0467, 1.4034, 1.4218, 1.3782, 1.5177], dtype=torch.float64)

In [21]:
x_t = x.transpose(-2, 1)
x_t.shape

torch.Size([5, 545])

In [22]:
x_t[1][:]

tensor([ 1.4034,  1.4034,  0.0473,  1.4034,  1.4034,  0.0473,  1.4034,
         2.7596,  1.4034,  0.0473,  0.0473,  1.4034,  1.4034,  1.4034,
         0.0473,  1.4034,  1.4034,  0.0473,  0.0473,  0.0473,  0.0473,
         0.0473,  0.0473,  0.0473,  0.0473,  1.4034,  0.0473,  0.0473,
         2.7596,  1.4034,  0.0473,  0.0473,  1.4034,  0.0473,  2.7596,
         0.0473,  0.0473,  1.4034,  0.0473,  1.4034,  0.0473,  0.0473,
         0.0473,  1.4034,  1.4034,  0.0473,  0.0473,  0.0473,  0.0473,
         0.0473,  0.0473,  0.0473,  1.4034,  0.0473,  0.0473,  0.0473,
         1.4034,  1.4034,  1.4034,  0.0473,  0.0473, -1.3089,  1.4034,
         1.4034,  0.0473,  0.0473, -1.3089,  0.0473,  0.0473,  1.4034,
         0.0473,  1.4034,  0.0473, -1.3089,  0.0473,  1.4034,  0.0473,
         0.0473,  0.0473,  0.0473,  0.0473,  0.0473,  0.0473,  0.0473,
         0.0473,  0.0473,  0.0473,  0.0473,  0.0473,  2.7596,  0.0473,
        -1.3089,  0.0473,  0.0473,  1.4034,  0.0473,  0.0473,  0.0473,
      

In [23]:
x_t[:][1]

tensor([ 1.4034,  1.4034,  0.0473,  1.4034,  1.4034,  0.0473,  1.4034,
         2.7596,  1.4034,  0.0473,  0.0473,  1.4034,  1.4034,  1.4034,
         0.0473,  1.4034,  1.4034,  0.0473,  0.0473,  0.0473,  0.0473,
         0.0473,  0.0473,  0.0473,  0.0473,  1.4034,  0.0473,  0.0473,
         2.7596,  1.4034,  0.0473,  0.0473,  1.4034,  0.0473,  2.7596,
         0.0473,  0.0473,  1.4034,  0.0473,  1.4034,  0.0473,  0.0473,
         0.0473,  1.4034,  1.4034,  0.0473,  0.0473,  0.0473,  0.0473,
         0.0473,  0.0473,  0.0473,  1.4034,  0.0473,  0.0473,  0.0473,
         1.4034,  1.4034,  1.4034,  0.0473,  0.0473, -1.3089,  1.4034,
         1.4034,  0.0473,  0.0473, -1.3089,  0.0473,  0.0473,  1.4034,
         0.0473,  1.4034,  0.0473, -1.3089,  0.0473,  1.4034,  0.0473,
         0.0473,  0.0473,  0.0473,  0.0473,  0.0473,  0.0473,  0.0473,
         0.0473,  0.0473,  0.0473,  0.0473,  0.0473,  2.7596,  0.0473,
        -1.3089,  0.0473,  0.0473,  1.4034,  0.0473,  0.0473,  0.0473,
      

## Split into train and validation sets

In [24]:
n_samples = x.shape[0]
n_val = int(percent_for_validation * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

print(f"training:\n{train_indices}")
print()
print(f"validation:\n{val_indices}")

training:
tensor([492, 460, 475,  58, 195, 100,  62,  49, 418,  50, 461, 107, 152,
        325, 361,  12, 335, 208,  99, 318, 110, 313,  48, 255, 346, 296,
        153, 493, 275,  28, 139, 439, 145,  31, 211, 309, 420, 210, 310,
        264, 254,  27, 341, 333, 423,  75, 470, 339, 451, 359, 124,  92,
        322, 342, 307,  70, 426, 223, 168, 134, 514,  40, 265, 408, 449,
         43, 457, 454, 218,  90, 165,  11, 435,  66, 170, 126, 422, 474,
        161,  17,  26, 127, 159,  39, 259, 424, 405, 404, 276, 174, 503,
        116, 354, 331,  54, 239, 489, 181, 458, 192, 441,  57,  96, 444,
        520, 315, 294, 433, 447, 391, 466, 212, 390, 541, 101, 443, 529,
        235, 498, 488, 355, 353,  76, 287, 251, 378, 149, 483, 256,  29,
        250, 131, 132, 522, 204, 527, 274, 365,  18, 291, 263, 122, 196,
        311, 367, 320, 510, 505,  72, 356, 327, 120, 289, 384, 432, 299,
         21, 531, 388, 284,  65,  86, 312, 515, 249, 244, 485, 409, 537,
        369, 180,  60,  80, 260,   1, 511

In [25]:
training_x = x[train_indices]
training_y  = y [train_indices]

validation_x = x[val_indices]
validation_y  = y[val_indices]

# training_un   = 0.1 * training_feature
# validation_un = 0.1 * validation_feature

## Define the training

In [26]:
def training_loop(n_epochs, optimizer, params, 
                  train_x, val_x,
                  train_y, val_y):
    for epoch in range(1, n_epochs + 1):
        train_predicted = model(train_x.transpose(-2, 1), *params)
        train_loss = loss_function(train_predicted, train_y)

        # For the valudation step
        with torch.no_grad(): 
            val_predicted = model(val_x.transpose(-2, 1), *params)
            val_loss = loss_function(val_predicted, val_y)
            assert val_loss.requires_grad == False # <2>
            
        optimizer.zero_grad()
        
        train_loss.backward()
        optimizer.step()
        
        if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.5f},"
                  f" Validation loss {val_loss.item():.5f}")

# Results

In [27]:
for rate in rates_to_learn_at:
    print(f"  Learning rate is {rate} :\n"+"/\\"*15)
    
    params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)

    optimizer = optim.SGD([params], lr=rate)

    training_loop(
        n_epochs = 5000, 
        optimizer = optimizer,
        params = params,
        train_x = training_x, 
        val_x = validation_x, 
        train_y = training_y,
        val_y = validation_y
    )
    
    val_predicted = model(validation_x.transpose(-2, 1), *params)
    val_loss = loss_function(val_predicted, validation_y)
    
    print(f"\nLoss:\t\t{val_loss:.5f}")
    print(f"Paramaters:\t{params.detach().numpy()}")
        
    print('\n'+"--"*30+'\n')

  Learning rate is 0.2 :
/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\
Epoch 1, Training loss 5.74220, Validation loss 7.31603
Epoch 2, Training loss 0.83596, Validation loss 1.02890
Epoch 3, Training loss 0.47635, Validation loss 0.53119
Epoch 500, Training loss 0.42980, Validation loss 0.48052
Epoch 1000, Training loss 0.42980, Validation loss 0.48052
Epoch 1500, Training loss 0.42980, Validation loss 0.48052
Epoch 2000, Training loss 0.42980, Validation loss 0.48052
Epoch 2500, Training loss 0.42980, Validation loss 0.48052
Epoch 3000, Training loss 0.42980, Validation loss 0.48052
Epoch 3500, Training loss 0.42980, Validation loss 0.48052
Epoch 4000, Training loss 0.42980, Validation loss 0.48052
Epoch 4500, Training loss 0.42980, Validation loss 0.48052
Epoch 5000, Training loss 0.42980, Validation loss 0.48052

Loss:		0.48052
Paramaters:	[ 0.17788048  0.2369681   0.34198764  0.07096225  0.3657186  -0.00676401]

------------------------------------------------------------

  Learning rate is 0.1

In [28]:
for rate in rates_to_learn_at:
    print(f"  Learning rate is {rate} :\n"+"/\\"*15)
    
    params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)

    optimizer = optim.Adam([params], lr=rate)

    training_loop(
        n_epochs = 5000, 
        optimizer = optimizer,
        params = params,
        train_x = training_x, 
        val_x = validation_x, 
        train_y = training_y,
        val_y = validation_y
    )
    
    val_predicted = model(validation_x.transpose(-2, 1), *params)
    val_loss = loss_function(val_predicted, validation_y)
    
    print(f"\nLoss:\t\t{val_loss:.5f}")
    print(f"Paramaters:\t{params.detach().numpy()}")
        
    print('\n'+"--"*30+'\n')

  Learning rate is 0.2 :
/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\
Epoch 1, Training loss 5.74220, Validation loss 7.31603
Epoch 2, Training loss 3.37011, Validation loss 4.34163
Epoch 3, Training loss 1.69083, Validation loss 2.16380
Epoch 500, Training loss 0.42980, Validation loss 0.48052
Epoch 1000, Training loss 0.42980, Validation loss 0.48052
Epoch 1500, Training loss 0.42980, Validation loss 0.48052
Epoch 2000, Training loss 0.42980, Validation loss 0.48052
Epoch 2500, Training loss 0.42981, Validation loss 0.48079
Epoch 3000, Training loss 0.42980, Validation loss 0.48052
Epoch 3500, Training loss 0.42980, Validation loss 0.48052
Epoch 4000, Training loss 0.42983, Validation loss 0.47990
Epoch 4500, Training loss 0.42980, Validation loss 0.48043
Epoch 5000, Training loss 0.42980, Validation loss 0.48052

Loss:		0.48052
Paramaters:	[ 0.17787504  0.23696269  0.34198216  0.07095686  0.3657132  -0.00675867]

------------------------------------------------------------

  Learning rate is 0.1